In [32]:
import twitter
import os
import pandas as pd
import time
from pprint import pprint
from itertools import chain
pjoin = os.path.join

In [2]:
# Get access tokens
tokens = {}
token_dir = '.tokens'
token_file = pjoin(token_dir, os.listdir('.tokens')[0])

with open(token_file, 'r') as f:
    for line in f.readlines():
        tag, token = line.split(': ')
        tokens[tag] = token.replace('\n', '')

# Initialize API instance
twitter_api = twitter.Api(
        consumer_key=tokens['api_key'],
        consumer_secret=tokens['api_secret_key'],
        access_token_key=tokens['access_token'],
        access_token_secret=tokens['access_token_secret']
    )

print(twitter_api.VerifyCredentials())

{"created_at": "Sat Aug 24 08:24:50 +0000 2019", "default_profile": true, "description": "Physicist, interested in software, tech, coffee & all that stuff | Boston University Physics PhD Student | Bo\u011fazi\u00e7i University \u201818", "favourites_count": 15, "followers_count": 13, "friends_count": 54, "id": 1165178081994248192, "id_str": "1165178081994248192", "location": "Boston, MA", "name": "Alp Akp\u0131nar", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1165178556328087552/avXATya5_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1165178556328087552/avXATya5_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "alp_akpinar"}


In [18]:
def build_test_set(keyword, num_tweets=1000):
    '''Build test set of tweets for a given keyword.'''
    all_tweets = []
    num_tweets_fetched = 0
    num_requests = int(num_tweets / 100)
    for _ in range(num_requests):
        print(f'Getting tweets: {num_tweets_fetched}/{num_tweets}', end='\r')
        tweets_fetched = twitter_api.GetSearch(keyword, count=100)
        all_tweets.append(tweets_fetched)
        
        num_tweets_fetched += 100
        
        # Wait 1 second until the next GET request
        time.sleep(1)

    print(f'Getting tweets: {num_tweets_fetched}/{num_tweets}')
        
    return all_tweets

# Build the test set for the sentiment classifier
test_set = build_test_set('Brexit')

Getting tweets: 1000/1000


In [19]:
# Flatten the test set
test_set = list(chain.from_iterable(test_set))

In [40]:
def build_training_set(corpus_file, cleaned_file):
    '''
    Clean the original CSV file (corpus_file), save results to new CSV file (cleaned_file)
    '''
    # Get contents from original CSV file first
    colnames = ['Sentiment', 'Tweet ID', 'DateTime', 'Query', 'Username', 'Text']
    df = pd.read_csv(corpus_file, encoding='latin-1', names=colnames)
    
    # Keep sentiment and text columns only
    clean_df = df[['Sentiment', 'Text']]

    # Save the cleaned df to new file
    clean_df.to_csv(cleaned_file, index=False)
    
    print(f'File saved: {cleaned_file}')
    
corpus_file = './data/tweet/sentiment_analysis/training_set.csv'
tweet_data_file = './data/tweet/sentiment_analysis/training_set_clean.csv'

build_training_set(corpus_file, tweet_data_file)


File saved: ./data/tweet/sentiment_analysis/training_set_clean.csv
